### HW1： 

### import libraries

In [61]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
# Cleaning the texts
import re
import nltk
nltk.download('stopwords') # dictionary contains all stopping words
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mengh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [62]:
econ_path = 'C:/Users/mengh/Dropbox/FRE-7871-News-Analytics-and-Machine-Learning/HW1/economist/'
onion_path = 'C:/Users/mengh/Dropbox/FRE-7871-News-Analytics-and-Machine-Learning/HW1/onion/'

def data_preparation(path, class_name):

    # Create Dictionary for File Name and Text
    file_name_and_text = {}
    
    if class_name == 'econ':
        sub_path_names = os.listdir(path)

        for sub_path in sub_path_names:
            file_names = os.listdir(path+sub_path)
            for file in file_names:
                if file.endswith(".txt"):
                    with open(path + sub_path + '/'+ file, "r") as target_file:
                         file_name_and_text[file] = target_file.read()
    else:
        file_names = os.listdir(path)

        for file in file_names:
            if file.endswith(".txt"):
                with open(path + file, "r") as target_file:
                     file_name_and_text[file] = target_file.read()

    file_data = (pd.DataFrame.from_dict(file_name_and_text, orient='index')
                 .reset_index().rename(index = str, columns = {'index': 'file_name', 0: 'text'}))
    file_data['class'] = class_name
    return file_data

def data_generate_label(econ_df, onion_df):
    df =  pd.concat([econ_df, onion_df], ignore_index=True)
    le = LabelEncoder()
    df['class'] = le.fit_transform(df['class'])
    df = df.astype({'class': int})
    
    return df 

econ_df = data_preparation(econ_path, 'econ')
onion_df = data_preparation(onion_path, 'oni')
article_df = data_generate_label(econ_df, onion_df)
article_df.head()

,file_name,text,class
0,africa.1.txt,\n ...,0
1,africa.10.txt,\n ...,0
2,africa.11.txt,\n The Kurds say they still want more of it ...,0
3,africa.12.txt,\n A politician who paid the price for telli...,0
4,africa.13.txt,\n President Mwai Kibaki is loth to back dow...,0


In [ ]:
def bag_of_word(article_df):
    corpus = [] # bag of words
    
    for index, row in article_df.iterrows():
        article = re.sub('[^a-zA-Z]', ' ', row['text']) # keep only letters
        article = article.lower() # lower case 
        article = article.split() # string to list 
        ps = PorterStemmer() # sstemmed

        article = [ps.stem(word) for word in article if not word in set(stopwords.words('english'))] #如果不在虚词字典，保留单词
        article = ' '.join(article) # list 转字符串
        corpus.append(article)
    
    return corpus


corpus = bag_of_word(article_df)
corpus

In [25]:
print(file_data.loc['0', 'text'])


                                                                  Reuters

                                  Reuters

   Are the latest terrorist bombs in north Africa home-grown, inspired by
   al-Qaeda or actually organised by it?


   THE bombs that shook Algiers and ripped the façade off the prime
   minister's office this week, killing some 30 people, suggest a
   worrying resurgence of the country's Islamist militants since they
   rebranded themselves as al-Qaeda's official arm in the Maghreb. This
   week's counter-attack on bombers in Morocco raises similar questions.
   But they were probably different sorts of terrorists. There is no
   presumption yet that al-Qaeda has effectively activated a joint north
   African front, though it would surely like to do so.

   It is the first time in years that Algeria's capital has been attacked
   on such a scale. Earlier this year there were terrorist shootings in
   Tunisia. In Morocco this week four suicide bombers died in Casablan

In [34]:
import re

file_1 = re.sub('[^a-zA-Z]', ' ', file_data['text'][0])
print(file_1)


                                                                   Reuters                                    Reuters     Are the latest terrorist bombs in north Africa home grown  inspired by    al Qaeda or actually organised by it       THE bombs that shook Algiers and ripped the fa ade off the prime    minister s office this week  killing some    people  suggest a    worrying resurgence of the country s Islamist militants since they    rebranded themselves as al Qaeda s official arm in the Maghreb  This    week s counter attack on bombers in Morocco raises similar questions     But they were probably different sorts of terrorists  There is no    presumption yet that al Qaeda has effectively activated a joint north    African front  though it would surely like to do so      It is the first time in years that Algeria s capital has been attacked    on such a scale  Earlier this year there were terrorist shootings in    Tunisia  In Morocco this week four suicide bombers died in Casablan

In [35]:
file_1 = file_1.lower()
file_1 = file_1.split()

In [33]:
file_1

['Reuters',
 'Reuters',
 'Are',
 'the',
 'latest',
 'terrorist',
 'bombs',
 'in',
 'north',
 'Africa',
 'home',
 'grown',
 'inspired',
 'by',
 'al',
 'Qaeda',
 'or',
 'actually',
 'organised',
 'by',
 'it',
 'THE',
 'bombs',
 'that',
 'shook',
 'Algiers',
 'and',
 'ripped',
 'the',
 'fa',
 'ade',
 'off',
 'the',
 'prime',
 'minister',
 's',
 'office',
 'this',
 'week',
 'killing',
 'some',
 'people',
 'suggest',
 'a',
 'worrying',
 'resurgence',
 'of',
 'the',
 'country',
 's',
 'Islamist',
 'militants',
 'since',
 'they',
 'rebranded',
 'themselves',
 'as',
 'al',
 'Qaeda',
 's',
 'official',
 'arm',
 'in',
 'the',
 'Maghreb',
 'This',
 'week',
 's',
 'counter',
 'attack',
 'on',
 'bombers',
 'in',
 'Morocco',
 'raises',
 'similar',
 'questions',
 'But',
 'they',
 'were',
 'probably',
 'different',
 'sorts',
 'of',
 'terrorists',
 'There',
 'is',
 'no',
 'presumption',
 'yet',
 'that',
 'al',
 'Qaeda',
 'has',
 'effectively',
 'activated',
 'a',
 'joint',
 'north',
 'African',
 'front

In [38]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords # 虚词字典
file_1 = [ps.stem(word) for word in file_1 if not word in set(stopwords.words('english'))]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mengh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: name 'ps' is not defined